# Reading Avro Blobs Into Parquet Data Sets

### Dependency Importing and Environment Variable Retrieval

In [1]:
import os
import string
import json
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession, SQLContext, Window
from pyspark.sql.functions import udf, mean, lit, stddev, col, expr, when, date_sub, avg, window
from pyspark.sql.types import DoubleType, ArrayType, ShortType, LongType, IntegerType, TimestampType, StructType, StringType, StructField
from pyspark.storagelevel import StorageLevel
from azureml.api.schema.dataTypes import DataTypes
from azureml.api.schema.sampleDefinition import SampleDefinition
from azureml.api.realtime.services import generate_schema
from azure.storage.table import TableService, Entity, TablePermissions

STORAGE_ACCOUNT_SUFFIX = 'core.windows.net'
TELEMETRY_STORAGE_ACCOUNT_NAME = os.getenv('TELEMETRY_STORAGE_ACCOUNT_NAME')
TELEMETRY_STORAGE_ACCOUNT_KEY = os.getenv('TELEMETRY_STORAGE_ACCOUNT_KEY')
TABLE_STORAGE_ACCOUNT_NAME = os.getenv('TABLE_STORAGE_ACCOUNT_NAME') #TODO need to add to ENV Variables
TABLE_STORAGE_ACCOUNT_KEY = os.getenv('TABLE_STORAGE_ACCOUNT_KEY') #TODO need to add to ENV Variables
TELEMETRY_CONTAINER_NAME = os.getenv('TELEMETRY_CONTAINER_NAME')
#LOGS_ARCHIVE_CONTAINER_NAME = 'logs-archive' #TODO: introduce environment variables
STAGING_STORAGE_ACCOUNT_NAME = os.getenv('STAGING_STORAGE_ACCOUNT_NAME')
STAGING_STORAGE_ACCOUNT_KEY = os.getenv('STAGING_STORAGE_ACCOUNT_KEY')
LOG_TABLE_NAME = os.getenv('LOG_TABLE_NAME') #TODO need to add this to ENV Variables

#### Development Variables

In [9]:
#For development purposes only until ENV Variables get set
STORAGE_ACCOUNT_SUFFIX = 'core.windows.net'
TELEMETRY_STORAGE_ACCOUNT_NAME = "stguwsjc4vojbrbw"
TELEMETRY_STORAGE_ACCOUNT_KEY = "2GGDWVBtGBy+hAgl5a1uGT4NeU2zzFdocuDFKnOwR2vc5wEOP7jTfbS3/Nl5vvzEudJ4nfH6ozmSOSPXo3xETA=="
TABLE_STORAGE_ACCOUNT_NAME = "stgmosxhnn5t3nyc"
TABLE_STORAGE_ACCOUNT_KEY = "fa0TlK+KYvXfLXGqfqVATbNF4xB5o64O4i7PjmlGMSTXCLMIOh/9Sc4ScpJ2V0vUzQ2TlK4wVu0BbA9i3HZGaw=="
TELEMETRY_CONTAINER_NAME = "telemetry"
LOG_TABLE_NAME = 'logs'
LOGS_ARCHIVE_CONTAINER_NAME = "logs" #TODO: introduce environment variables
STAGING_STORAGE_ACCOUNT_NAME = "stguwsjc4vojbrbw"
STAGING_STORAGE_ACCOUNT_KEY = "2GGDWVBtGBy+hAgl5a1uGT4NeU2zzFdocuDFKnOwR2vc5wEOP7jTfbS3/Nl5vvzEudJ4nfH6ozmSOSPXo3xETA=="


### Setting up Drop Folder

In [3]:
from pathlib import Path
data_dir = str(Path.home()) + '/data'

#TODO: Convert data_dir into env variable
% rm -rf $data_dir
% mkdir $data_dir $data_dir/logs

### Retrieving telemetry data (as spark dataframe)

In [4]:
wasbTelemetryUrl = "wasb://{0}@{1}.blob.{2}/*/*/*/*/*/*/*".format(TELEMETRY_CONTAINER_NAME, 
                                                                  TELEMETRY_STORAGE_ACCOUNT_NAME, 
                                                                  STORAGE_ACCOUNT_SUFFIX)

sc = SparkSession.builder.getOrCreate()
hc = sc._jsc.hadoopConfiguration()
hc.set("avro.mapred.ignore.inputs.without.extension", "false")
if TELEMETRY_STORAGE_ACCOUNT_KEY:
     hc.set("fs.azure.account.key.{}.blob.core.windows.net".format(TELEMETRY_STORAGE_ACCOUNT_NAME), TELEMETRY_STORAGE_ACCOUNT_KEY)
hc.set("fs.azure.account.key.{}.blob.core.windows.net"
    .format(STAGING_STORAGE_ACCOUNT_NAME), STAGING_STORAGE_ACCOUNT_KEY)
sql = SQLContext.getOrCreate(sc)
avroblob = sql.read.format("com.databricks.spark.avro").load(wasbTelemetryUrl)
avroblob.show()

+--------------------+----------+--------------------+--------------------+
|     EnqueuedTimeUtc|Properties|    SystemProperties|                Body|
+--------------------+----------+--------------------+--------------------+
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T

### Convert byteformatted "body" of raw blob data into JSON, then explode result into new Pyspark DataFrame

In [5]:
#Convert byteformat to string format in pyspark dataframe
from json import loads as Loads
column = avroblob['Body']
string_udf = udf(lambda x: x.decode("utf-8"))
avroblob=avroblob.withColumn("BodyString", string_udf(column))
avroblob.printSchema()

#Convert "body" into new DataFrame
telemetry_df = sql.read.json(avroblob.select("BodyString").rdd.map(lambda r: r.BodyString))
telemetry_df.show()

root
 |-- EnqueuedTimeUtc: string (nullable = true)
 |-- Properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- SystemProperties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- Body: binary (nullable = true)
 |-- BodyString: string (nullable = true)

+----------------+-------------------+-----------+--------+-------+-------------+-----------+--------------------+---------+
|ambient_pressure|ambient_temperature|  machineID|pressure|  speed|speed_desired|temperature|           timestamp|vibration|
+----------------+-------------------+-----------+--------+-------+-------------+-----------+--------------------+---------+
|          100.92|              20.06|Machine-004|  655.58|1133.27|         1000|     174.62|2018-06-20T16:41:...|     null|
|          101.02|              19.95|Machine-001|   46.46|  -2.17|            0|     164.84|2018-06-20T16:41:...|     null|
|          101

In [6]:
#columns to retain: timestamp, ambient_pressure, ambient_temperature machineID, pressure, speed, 
#                   speed_desired, temperature
subsetted_df = telemetry_df.select(["timestamp", "ambient_pressure","ambient_temperature","machineID","pressure","speed","speed_desired","temperature"])


In [7]:
#modify timestamp format
import datetime
e = '%Y-%m-%dT%H:%M:%S.%f'
timestamp_udf = udf(lambda date: datetime.datetime.strptime(date, e), TimestampType())
reformatted_time_df = subsetted_df.withColumn("timestamp", timestamp_udf(subsetted_df["timestamp"]))

reformatted_time_df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- ambient_pressure: double (nullable = true)
 |-- ambient_temperature: double (nullable = true)
 |-- machineID: string (nullable = true)
 |-- pressure: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- speed_desired: long (nullable = true)
 |-- temperature: double (nullable = true)



### Write dataframe to Parquet in system storage

In [10]:
reformatted_time_df.write.parquet(data_dir+"/telemetry", mode="overwrite")

## Get Logs

In [11]:
#table retrieval
table_service = TableService(account_name=TABLE_STORAGE_ACCOUNT_NAME, account_key=TABLE_STORAGE_ACCOUNT_KEY)
tblob = table_service.query_entities(LOG_TABLE_NAME)

### Process log table data into Pandas DataFrame

In [12]:
attributes = list()
for row in tblob:
    if (len(attributes) == 0):
        for attribute in row:
            attributes.append(attribute)
    break
log_df = pd.DataFrame(columns=attributes)
for row in tblob:
    row_dict = {}    
    for attribute in row:
        if (attribute != "Timestamp"):
            row_dict[attribute] = row[attribute]
        else:
            newtime = row[attribute].replace(tzinfo=None)
            timeitem = pd.Timestamp(newtime, tz=None)
            row_dict[attribute] = timeitem
    log_df = log_df.append(row_dict, ignore_index=True)
log_df.head()

,Level,PartitionKey,Message,Code,Timestamp,etag,RowKey,_Driver
0,DEBUG,Machine-000,"{""h1"": 0.9374248680712486, ""h2"": 0.92468197881...",SIM_HEALTH,2018-06-25 13:21:01.766912,"W/""datetime'2018-06-25T13%3A21%3A01.7669126Z'""",008e47d2214d4d29a1cecec93bc8d6ef,46132c63-8f3a-4fc7-acf8-50a5d8fa94ae
1,DEBUG,Machine-000,"{""h1"": 0.9376092354171048, ""h2"": 0.92541503564...",SIM_HEALTH,2018-06-25 09:15:49.483992,"W/""datetime'2018-06-25T09%3A15%3A49.483992Z'""",024fb3d61ebf45a28d369fa0680b7941,46132c63-8f3a-4fc7-acf8-50a5d8fa94ae
2,DEBUG,Machine-000,"{""h1"": 0.9372548374119425, ""h2"": 0.92398388622...",SIM_HEALTH,2018-06-25 17:30:44.172884,"W/""datetime'2018-06-25T17%3A30%3A44.1728843Z'""",0259e7d02bc04c65aeec4d5bbd4fcd9f,46132c63-8f3a-4fc7-acf8-50a5d8fa94ae
3,DEBUG,Machine-000,"{""h1"": 0.9375956117653576, ""h2"": 0.92536168736...",SIM_HEALTH,2018-06-25 09:33:56.598835,"W/""datetime'2018-06-25T09%3A33%3A56.5988351Z'""",079f69861a314898b76f540ca0fe9fe8,46132c63-8f3a-4fc7-acf8-50a5d8fa94ae
4,DEBUG,Machine-000,"{""h1"": 0.9373586393360502, ""h2"": 0.92441264690...",SIM_HEALTH,2018-06-25 14:48:46.091508,"W/""datetime'2018-06-25T14%3A48%3A46.0915087Z'""",08880c1fa6eb4844a161aab941a821a9,46132c63-8f3a-4fc7-acf8-50a5d8fa94ae


### Select necessary attributes

In [13]:
log_df = log_df[["Timestamp", "Code", "Level", "PartitionKey"]]
log_df.columns = ["timestamp", "code","level","machineID"]
log_df.index = log_df['timestamp']
del log_df['timestamp']
log_df.head()

,code,level,machineID
timestamp,,,
2018-06-25 13:21:01.766912,SIM_HEALTH,DEBUG,Machine-000
2018-06-25 09:15:49.483992,SIM_HEALTH,DEBUG,Machine-000
2018-06-25 17:30:44.172884,SIM_HEALTH,DEBUG,Machine-000
2018-06-25 09:33:56.598835,SIM_HEALTH,DEBUG,Machine-000
2018-06-25 14:48:46.091508,SIM_HEALTH,DEBUG,Machine-000


### Write logs to system storage

In [14]:
log_df.to_parquet(data_dir+"/logs/logs.parquet", engine='fastparquet', times='int96')